<a href="https://colab.research.google.com/github/Raiden-Makoto/BQML-Projects/blob/main/NCAA_Bracketology/EDA/NCAA_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring NCAA Data with BigQuery

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import userdata
project = userdata.get('PROJ_NAME')

In [3]:
from google.cloud import bigquery
# Replace 'your-gcp-project-id' with your actual Google Cloud Project ID.
# You can find your project ID in the Google Cloud Console.
# Explicitly set the location to 'US' to access the public dataset.
client = bigquery.Client(project=project, location='US')

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

## Task 2: Writing Queries

In [5]:
query = """
#standardSQL
SELECT event_type, COUNT(*) AS event_count
FROM `bigquery-public-data.ncaa_basketball.mbb_pbp_sr`
GROUP BY 1
ORDER BY event_count DESC
"""
df = client.query(query).to_dataframe()
df.head()

,event_type,event_count
0,rebound,790490
1,twopointmiss,425422
2,turnover,410166
3,twopointmade,400648
4,freethrowmade,316637


Looking at your results, how many historical shots were TWOPOINTMADE or FREETHROWMISS?

In [6]:
twopointmade_count = df[df['event_type'] == 'twopointmade']['event_count'].iloc[0]
print(f"Number of 'twopointmade' events: {twopointmade_count}")

Number of 'twopointmade' events: 400648


In [8]:
freethrowmiss_count = df[df['event_type'] == 'freethrowmiss']['event_count'].iloc[0]
print(f"Number of 'freethrowmiss' events: {freethrowmiss_count}")

Number of 'freethrowmiss' events: 134256


## Task 3: Fun Queries

Which 5 games featured the most three point shots made? How accurate were all the attempts?

In [9]:
query = """
#standardSQL
SELECT
    scheduled_date,
    name,
    market,
    alias,
    three_points_att,
    three_points_made,
    three_points_pct,
    opp_name,
    opp_market,
    opp_alias,
    opp_three_points_att,
    opp_three_points_made,
    opp_three_points_pct,
    (three_points_made + opp_three_points_made) AS total_threes
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE season > 2010
ORDER BY total_threes DESC
LIMIT 5;
"""
df = client.query(query).to_dataframe()
df.head()

,scheduled_date,name,market,alias,three_points_att,three_points_made,three_points_pct,opp_name,opp_market,opp_alias,opp_three_points_att,opp_three_points_made,opp_three_points_pct,total_threes
0,2016-11-18,Knights,Middle Georgia,MGC,38,16,42.1,Tigers,Savannah State,SAV,52,24,46.2,40
1,2016-11-18,Tigers,Savannah State,SAV,52,24,46.2,Knights,Middle Georgia,MGC,38,16,42.1,40
2,2016-11-22,Wildcats,Fort Valley State,FVSU,39,14,35.9,Tigers,Savannah State,SAV,45,23,51.1,37
3,2016-11-22,Tigers,Savannah State,SAV,45,23,51.1,Wildcats,Fort Valley State,FVSU,39,14,35.9,37
4,2017-12-10,Cougars,Cincinnati Clermont,CINC,51,21,41.2,Skyhawks,Tennessee-Martin,UTM,35,15,42.9,36


We can see that the Tigers made over $50\%$ of their three-pointers on 11-22-2016. Wow!

Now we want to know which 5 basketball venues have the highest seating capacity

In [10]:
query = """
#standardSQL
SELECT
    venue_name,
    venue_capacity,
    venue_city,
    venue_state
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
GROUP BY 1,2,3,4
ORDER BY venue_capacity DESC
LIMIT 5;
"""
df = client.query(query).to_dataframe()
df.head()


,venue_name,venue_capacity,venue_city,venue_state
0,AT&T Stadium,80000,Arlington,TX
1,University of Phoenix Stadium,72220,Glendale,AZ
2,NRG Stadium,71054,Houston,TX
3,Georgia Dome,71000,Atlanta,GA
4,Lucas Oil Stadium,70000,Indianapolis,IN


We are also curious which game was the highest-scoring (total number of points by both teams)

In [11]:
query = """
#standardSQL
SELECT scheduled_date,
  name,
  market,
  alias,
  points_game AS team_points,
  opp_name,
  opp_market,
  opp_alias,
  opp_points_game AS opposing_team_points,
  points_game + opp_points_game AS point_total
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE season > 2010
ORDER BY point_total DESC
LIMIT 5;
"""
df = client.query(query).to_dataframe()
df.head()

,scheduled_date,name,market,alias,team_points,opp_name,opp_market,opp_alias,opposing_team_points,point_total
0,2017-02-10,Terriers,Wofford,WOF,131,Bulldogs,Samford,SAM,127,258
1,2017-02-10,Bulldogs,Samford,SAM,127,Terriers,Wofford,WOF,131,258
2,2017-02-04,Vikings,Portland State,PRST,124,Eagles,Eastern Washington,EWU,130,254
3,2017-02-04,Eagles,Eastern Washington,EWU,130,Vikings,Portland State,PRST,124,254
4,2013-11-14,Crusaders,Holy Cross,HC,122,Pioneers,Sacred Heart,SHU,118,240


We see that in the Samford V Wofford game on 2017-02-10, a total of $131+127=258$ points were made!

Finally, what game resulted in the biggest victory/biggest defeat? That is, which games had the biggest point differential? We are interested in the finals games only because that's where the most fun happens.

In [12]:
query = """
#standardSQL
SELECT scheduled_date,
  name,
  market,
  alias,
  points_game AS team_points,
  opp_name,
  opp_alias,
  opp_points_game AS opposing_team_points,
  points_game - opp_points_game AS point_differential
FROM `bigquery-public-data.ncaa_basketball.mbb_teams_games_sr`
WHERE season > 2010 AND tournament_type = 'National Championship'
ORDER BY point_differential DESC
LIMIT 5;
"""
df = client.query(query).to_dataframe()
df.head()


,scheduled_date,name,market,alias,team_points,opp_name,opp_alias,opposing_team_points,point_differential
0,2018-04-03,Wildcats,Villanova,VILL,79,Wolverines,MICH,62,17
1,2014-04-08,Huskies,Connecticut,CONN,60,Wildcats,UK,54,6
2,2017-04-04,Tar Heels,North Carolina,UNC,71,Bulldogs,GONZ,65,6
3,2015-04-07,Blue Devils,Duke,DUKE,68,Badgers,WIS,63,5
4,2016-04-05,Wildcats,Villanova,VILL,77,Tar Heels,UNC,74,3


We see that in the Wildcats vs Wolverines game on 2018-04-03, the Wildcats defeated the Wolverines $79$ to $62$, a point differential of $17$!